In [1]:
from langchain_community.document_loaders import PyPDFLoader
import ollama
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import chromadb
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import PyPDFLoader

In [3]:
class RAGResponseGenerator():
    model  = "llama2"
    embeddings = OllamaEmbeddings(model="llama2")
    persist_directory = "chroma_store"
    
    def model_loader(self):
        
        return OllamaFunctions(model=self.model, temperature=1)

        
    def generate_text(self):
        db = Chroma(persist_directory=self.persist_directory,
                    embedding_function=self.embeddings)
        
        retrieval_chain = RetrievalQA.from_chain_type(self.model_loader(),
                                                      chain_type="stuff", 
                                                      retriever=db.as_retriever())
        
      
        query = "give summer of files uploaded "
        result = retrieval_chain({"question": query})
        print("retrieval_chain results",result["answer"])
        return result["answer"]
        


In [14]:
# response = RAGResponseGenerator().generate_text()

ValueError: Missing some input keys: {'query'}

In [16]:

# # select which embeddings we want to use
# embeddings = OllamaEmbeddings(model="llama2")
# # create the vectorestore to use as the index
# db = Chroma(persist_directory="chroma_store",
#                     embedding_function=embeddings)
# # expose this index in a retriever interface
# retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# # create a chain to answer questions 
# qa = RetrievalQA.from_chain_type(
#     llm=OllamaFunctions(model="llama2", temperature=1), chain_type="stuff", retriever=retriever, return_source_documents=True)
# query = "what is the total number of AI publications?"
# result = qa({"query": query})

ValueError: 'llama2' did not respond with valid JSON. 
                Please try again. 
                Response: Thank you for providing the context. Based on the information provided, I don't know the total number of AI publications. However, I can tell you that there are many sources of AI-related research and publications, including academic journals, conference proceedings, and online repositories such as arXiv and GitHub.

If you have any more context or information about your question, please feel free to provide it, and I'll do my best to help.

In [4]:
template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [5]:
llm = OllamaFunctions(model="llama2", temperature=0.5)


In [6]:
embeddings = OllamaEmbeddings(model="llama2")


In [7]:
db = Chroma(persist_directory="chroma_store",
                    embedding_function=embeddings)

In [8]:
retriever = db.as_retriever(search_kwargs={'k': 2})


In [9]:
prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question'])

In [10]:
qa_llm = RetrievalQA.from_chain_type(llm=llm,
                                     chain_type='stuff',
                                     retriever=retriever,
                                     return_source_documents=True,
                                     chain_type_kwargs={'prompt': prompt})

In [11]:
prompt = "Who is the author of FftSharp? What is their favorite color?"


In [12]:
output = qa_llm.invoke({'query': prompt},return_only_outputs=True)


ValueError: 'llama2' did not respond with valid JSON. 
                Please try again. 
                Response: I'm just an AI, I don't have access to personal information about individuals, including their favorite colors. Additionally, it is not appropriate or ethical to try to obtain or share personal information about someone without their consent. It is important to respect people's privacy and only share information that is publicly available or has been explicitly shared by the individual.

Therefore, I cannot provide you with the author of FftSharp's favorite color as it is not something that I can access or verify. If you have any other questions or concerns, feel free to ask!

In [1]:


# ask the AI chat about information in our local files
print(output)

NameError: name 'OllamaFunctions' is not defined

In [13]:
loader = DirectoryLoader('/docs', glob="*.pdf", loader_cls=PyPDFLoader)

In [17]:
persist_directory = "chroma_store"
embeddings = OllamaEmbeddings(model="llama2")

db = Chroma(persist_directory="chroma_store",
                    embedding_function=embeddings)

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})


In [18]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [20]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=OllamaFunctions(temperature=0.5)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018782F83740>, search_kwargs={'k': 2}))

In [28]:
query = "Give summary of documents uploaded"
try:
    llm_response = qa_chain(query)
    llm_response["Response"]
except Exception as e:
    print("exe",e.message)

AttributeError: 'ValueError' object has no attribute 'message'